In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

secret_key = '6c5a66576979756e33334452684245'
start_urls = "http://swopenAPI.seoul.go.kr/api/subway"
url = f'{start_urls}/{secret_key}/xml/realtimeStationArrival/all'

In [3]:
response = requests.get(url)

In [4]:
soup = BeautifulSoup(response.text, "xml") # 실시간 도착 정보 받기

In [5]:
soup

<?xml version="1.0" encoding="utf-8"?>
<realtimeStationArrival>
<RESULT>
<code>INFO-000</code>
<developerMessage/>
<link/>
<message>정상 처리되었습니다.</message>
<status>200</status>
<total>2968</total>
</RESULT>
<row>
<rowNum>1</rowNum>
<selectedCount>2968</selectedCount>
<totalCount>2968</totalCount>
<subwayId>1009</subwayId>
<updnLine>상행</updnLine>
<trainLineNm>개화행 - 당산방면</trainLineNm>
<statnFid>1009000915</statnFid>
<statnTid>1009000913</statnTid>
<statnId>1009000914</statnId>
<statnNm>국회의사당</statnNm>
<trnsitCo>1</trnsitCo>
<ordkey>01000개화0</ordkey>
<subwayList>1009</subwayList>
<statnList>1009000914</statnList>
<btrainSttus>일반</btrainSttus>
<barvlDt>20</barvlDt>
<btrainNo>9067</btrainNo>
<bstatnId>901</bstatnId>
<bstatnNm>개화</bstatnNm>
<recptnDt>2023-11-21 10:36:24</recptnDt>
<arvlMsg2>국회의사당 진입</arvlMsg2>
<arvlMsg3>국회의사당</arvlMsg3>
<arvlCd>0</arvlCd>
</row>
<row>
<rowNum>2</rowNum>
<selectedCount>2968</selectedCount>
<totalCount>2968</totalCount>
<subwayId>1009</subwayId>
<updnLine>상행</up

In [6]:
station_2_id = pd.read_excel('station_information.xlsx', engine='openpyxl')
station_2_id.head()

,SUBWAY_ID,STATN_ID,STATN_NM,호선이름
0,1001,1001000100,소요산,1호선
1,1001,1001000101,동두천,1호선
2,1001,1001000102,보산,1호선
3,1001,1001000103,동두천중앙,1호선
4,1001,1001000104,지행,1호선


In [7]:
station_2_id = station_2_id[station_2_id['호선이름']=='2호선']
station_2_name = station_2_id['STATN_NM']
station_2_id = station_2_id['SUBWAY_ID']

In [8]:
station_2_name

99            시청
100        을지로입구
101        을지로3가
102        을지로4가
103    동대문역사문화공원
104           신당
105         상왕십리
106          왕십리
107          한양대
108           뚝섬
109           성수
110         건대입구
111           구의
112           강변
113         잠실나루
114           잠실
115         잠실새내
116        종합운동장
117           삼성
118           선릉
119           역삼
120           강남
121           교대
122           서초
123           방배
124           사당
125          낙성대
126        서울대입구
127           봉천
128           신림
129          신대방
130      구로디지털단지
131           대림
132          신도림
133           문래
134        영등포구청
135           당산
136           합정
137         홍대입구
138           신촌
139           이대
140           아현
141          충정로
142           용답
143           신답
144           용두
145          신설동
146          도림천
147         양천구청
148        신정네거리
149          까치산
Name: STATN_NM, dtype: object

In [9]:
import xml.etree.ElementTree as ET

In [10]:
xml_data = soup.prettify()
root = ET.fromstring(xml_data)

In [11]:
# 각 <row> 태그의 데이터를 리스트로 추출
rows = []
for row_elem in root.findall('row'):
    row_data = {}
    for child in row_elem:
        row_data[child.tag] = child.text
    rows.append(row_data)

# 리스트를 판다스 DataFrame으로 변환
df = pd.DataFrame(rows)
df = df.applymap(lambda x: x.replace('\n', '').strip())
df

,rowNum,selectedCount,totalCount,subwayId,updnLine,trainLineNm,statnFid,statnTid,statnId,statnNm,...,statnList,btrainSttus,barvlDt,btrainNo,bstatnId,bstatnNm,recptnDt,arvlMsg2,arvlMsg3,arvlCd
0,1,2968,2968,1009,상행,개화행 - 당산방면,1009000915,1009000913,1009000914,국회의사당,...,1009000914,일반,20,9067,901,개화,2023-11-21 10:36:24,국회의사당 진입,국회의사당,0
1,2,2968,2968,1009,상행,개화행 - 둔촌오륜방면,1009000938,1009000937,1009000938,중앙보훈병원,...,1009000931,일반,20,9079,901,개화,2023-11-21 10:35:30,중앙보훈병원 도착,중앙보훈병원,1
2,3,2968,2968,1065,상행,검암행 - 검암방면,1065006507,1065006505,1065006506,계양,...,"1065006506,1069073110",일반,0,A3035,70,검암,2023-11-21 10:35:31,계양 도착,계양,1
3,4,2968,2968,1001,상행,광운대행 - 구로방면,1001080143,1001000141,1001080142,가산디지털단지,...,"1001080142,1007000746",일반,0,0440,171,광운대,2023-11-21 10:35:17,가산디지털단지 도착,가산디지털단지,1
4,5,2968,2968,1001,상행,광운대행 - 금천구청방면,1001080146,1001080144,1001080145,석수,...,1001080145,일반,0,0628,171,광운대,2023-11-21 10:34:47,석수 도착,석수,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2963,2964,2968,2968,1001,하행,인천행 - 송내방면,1001000148,1001000150,1001000149,중동,...,1001000149,일반,0,1605,97,인천,2023-11-21 10:34:25,[44]번째 전역 (덕정),덕정,99
2964,2965,2968,2968,1001,하행,인천행 - 부개방면,1001000149,1001000151,1001000150,송내,...,1001000150,일반,0,1605,97,인천,2023-11-21 10:34:25,[45]번째 전역 (덕정),덕정,99
2965,2966,2968,2968,1001,하행,인천행 - 부평방면,1001000150,1001000152,1001000151,부개,...,1001000151,일반,0,1605,97,인천,2023-11-21 10:34:25,[46]번째 전역 (덕정),덕정,99
2966,2967,2968,2968,1001,하행,인천행 - 백운방면,1001000151,1001000153,1001000152,부평,...,"1001000152,1069073120",일반,0,1605,97,인천,2023-11-21 10:34:25,[47]번째 전역 (덕정),덕정,99


In [12]:
filtered_df = df[df['statnNm'].isin(station_2_name)]

In [20]:
from datetime import datetime

# 현재 날짜와 시간을 'YYYYMMDD' 형식의 문자열로 변환
current_datetime = datetime.now().strftime('%Y%m%d_%H%M%S')

# 파일명에 날짜를 포함
filename = f'filtered_data_{current_datetime}.csv'

# CSV 파일로 저장
filtered_df.to_csv(filename, index=False)